# Model search pipeline

This notebook trains a supervised model via model search techniques, which tune hyperparameters using an algorithm rather than brute force.

Specifically,
* Use `hyperopt-sklearn` to search the model space
* Specify a sub-region of the search space, a particular model type

We finally test the model on the test data set, as defined by the authors.

Note: some improvements could be made with respect to the training process like what was done in the previous notebook.

In [1]:
%cd ..

/project


In [2]:
from src.data import *

## Prepare the data

Load the feature data and then prepare the train / test objects.

In [3]:
activities = load_activity_names(); activities
features_df = load_feature_data() \
    .merge(activities) \
    .drop('activity_id', axis=1) \
    .sort_values(['subject_id', 'time_window_s']) \
    .reset_index(drop=True)
features_df.shape

(7352, 564)

We only input the data features into the model, so we need to skip subject, time, and activity labels.

In [4]:
X_train = features_df.drop(['subject_id', 'time_window_s', 'activity_name'], axis=1)
y_train = features_df.activity_name

In [5]:
features_test_df = load_feature_data('test') \
    .merge(activities) \
    .drop('activity_id', axis=1) \
    .sort_values(['subject_id', 'time_window_s']) \
    .reset_index(drop=True)
features_test_df.shape

(2947, 564)

## Model search and evaluation

### Model set-up

In this example, we fix the model search space for only support vector classifier (SVC) models. Here, the type of model used is not as important as the search process that `hyperopt` is going through.

In [6]:
from hpsklearn import HyperoptEstimator, svc

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [7]:
model = HyperoptEstimator(classifier=svc('svc-only'))

### Training / searching

The `hpsklearn.HyperoptEstimator` works like other scikit-learn estimator objects in that it has a `.fit` and a `.predict` method.

In [8]:
model.fit(X_train, y_train)

100%|██████████| 10/10 [00:01<00:00,  1.16s/trial, best loss: 0.023113528212100665]


In [9]:
y_hat = model.predict(X_train)

In [10]:
from sklearn.metrics import accuracy_score, classification_report

We happen to get 100% accuracy on the training data set.

In [11]:
accuracy_score(y_hat, y_train)

1.0

### Evaluate on the test data

We evaluate the model on the test data that was defined by the authors.

In [12]:
y_test_hat = model \
    .predict(features_test_df.drop(['subject_id', 'time_window_s', 'activity_name'], axis=1))

In [13]:
print(classification_report(y_test_hat, features_test_df.activity_name))

                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       538
           SITTING       0.91      0.98      0.95       457
          STANDING       0.98      0.93      0.96       563
           WALKING       0.98      0.95      0.96       510
WALKING_DOWNSTAIRS       0.95      0.98      0.96       407
  WALKING_UPSTAIRS       0.94      0.94      0.94       472

          accuracy                           0.96      2947
         macro avg       0.96      0.96      0.96      2947
      weighted avg       0.96      0.96      0.96      2947



When we cross-tabulate the actual labels with the classified ones, we see a pretty diagonal matrix. Indeed, laying has been 100% correct.

Here we will want to validate if the errors made are acceptable. For example, errors for walking downstairs are either walking or walking upstairs. It may be important to continue tuning parameters such that this activity is never (or less commonly) misclassified as walking upstairs.

In [14]:
pd.crosstab(features_test_df.activity_name.values,
            y_test_hat,
            rownames=['True'],
            colnames=['Classified'])

Classified,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,1,449,39,0,0,2
STANDING,0,8,524,0,0,0
WALKING,0,0,0,485,6,5
WALKING_DOWNSTAIRS,0,0,0,3,397,20
WALKING_UPSTAIRS,0,0,0,22,4,445
